In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
import shap
from catboost import *
from sklearn.preprocessing import label_binarize
from catboost import *
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled_TOF_MCL.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Ionization Potential',#0
                      'Electronegativity',#1
                      'Number of d electrons',#2
                      'Graphene/Carbon Nanosheets or other 2D Structures',#3
                      'Carbon Nanofiber/Nanotubes',#4
                      'Biomass or other Organic Derived',#5  
                      'Main Transition Metal Content (wt. %)',#6
                      'Nitrogen Cotent (wt. %)',#7
                      'Metal-N Coordination Number (XAS)',#8    
                      'Pyridinic N Ratio',#9
                      'Pyrrolic N Ratio',#10
                      'Raman ID/IG Ratio',#11
                      'BET Surface Area (m2/g)',#12
                      'Pyrolysis Temperature (°C)',#13
                      'Pyrolysis Time (h)',#14
                      'Rising Rate (°C min-1)',#15
                      'Flow Cell/H-type Cell',#16
                      'Electrolyte Concentration (M)',#17
                      'Catalyst Loading (mg cm-2)',#18
                      'Carbon Paper/Glassy Carbon',#19
                      'Electrolyte pH',#20
                      'TOF_MCL_2_4'#21
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:21]
raw_power=raw_data.iloc[:,21]
print('ready')
nb_classes=3
print('ready')
from sklearn.metrics import roc_curve, auc


In [ ]:
def auc_curve(y_label, y_pre,algorithm_name):
#     y_label = y_label + 1
#     y_pre = y_pre + 1
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(nb_classes):
        fpr[i], tpr[i], _ = roc_curve(y_label[:, i], y_pre[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_label.ravel(), y_pre.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(nb_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(nb_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= nb_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])



    # Plot all ROC curves
    lw = 2
    fig=plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(nb_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Multi-Class Classification) of %s' %algorithm_name)
    plt.legend(loc="lower right")
    plt.savefig('ROC Curve of %s OVPMCL.png' %algorithm_name)
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-1)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    ##############################################################
    Y_test_labeled = label_binarize(y_test, classes=[i for i in range(nb_classes)])
    y_score=best_model.predict_proba(X_test)
#     y_score=y_score[:,1]
#     print(y_score)
    auc_curve(Y_test_labeled,y_score,algorithm_name)
    ##############################################################
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    print(final_result['accuracy'])
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
    return best_model
def shap_plot_interaction(best_model,algorithm_name,interacted_features):
    print(algorithm_name)
    SHAP_INPUT=raw_param
    SHAP_OUTPUT=raw_power
    print('train finished')
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)

    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        
        interaction_values = best_model.get_feature_importance(Pool(SHAP_INPUT),type=EFstrType.ShapInteractionValues)
        interaction_values=interaction_values[:,:-1,:-1]
        shap.dependence_plot(interacted_features[0], list(shap_values_reshaped[:,:,:-1])[2], SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], list(shap_values_reshaped[:,:,:-1])[2], SHAP_INPUT,interaction_index= interacted_features[0])
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)        
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT) 
        print(shap_values)
        shap.dependence_plot(interacted_features[0], shap_values[2], SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values[2], SHAP_INPUT,interaction_index= interacted_features[0])
    elif algorithm_name=='ANN':
        SHAP_INPUT=standardized_data.iloc[:,0:22]
        SHAP_OUTPUT=raw_data.iloc[:,22]
        X_SHAP=SHAP_INPUT.values.astype(np.float32)
        y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
        explainer = shap.GradientExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP)        
        shap.dependence_plot(interacted_features[0], shap_values[1], SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values[1], SHAP_INPUT,interaction_index= interacted_features[0])
    elif algorithm_name=='LightGBM':
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.dependence_plot(interacted_features[0], shap_values[2], SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values[2], SHAP_INPUT,interaction_index= interacted_features[0])
    else:
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT)
        shap.dependence_plot(interacted_features[0], shap_values[2], SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values[2], SHAP_INPUT,interaction_index= interacted_features[0])
from pdpbox import pdp
def plot_pdp_interact_ANN(model, df, f_list, cluster_flag=False, nb_clusters=None, lines_flag=False):
    
    # Create the data that we will plot
    inter1 = pdp.pdp_interact(model, df, model_features=df.columns.tolist(), features=f_list,num_grid_points=[100,100])
    # plot it
    settings = {
            'contour_color':  'white',
            'font_family': 'Arial',
            # matplotlib color map for interact plot
            'cmap': 'viridis',
            # fill alpha for interact plot
            'inter_fill_alpha': 0.8,
            # fontsize for interact plot text
            'inter_fontsize': 7,
        }
    pdp.pdp_interact_plot(
    pdp_interact_out=inter1, feature_names=f_list, plot_type='contour',figsize=(10,10),x_quantile=True, plot_pdp=True,plot_params=settings)
from sklearn.inspection import plot_partial_dependence
from sklearn.utils import validation
def pdp_plot_2d(best_model,f_list):
    print('start')
    validation.check_is_fitted(estimator=best_model)
    my_plots =plot_partial_dependence(best_model, features=[f_list], X=raw_param, percentiles=(0, 1),grid_resolution=100,target=2)
def pdp_plot_2d_XG_CAT(best_model,f_list):
    print('start')
    best_model.dummy_ = "dummy"
    validation.check_is_fitted(estimator=best_model)
    my_plots =plot_partial_dependence(best_model, features=[f_list], X=raw_param, percentiles=(0, 1),grid_resolution=100,target=2)
def pdp_plot_2d_ANN(model,f_list):
    print('start')
    model.dummy_ = "dummy"
    print(type(model))
    validation.check_is_fitted(estimator=model)
    my_plots =plot_partial_dependence(model, features=[f_listt], X=raw_input, percentiles=(0, 1))

In [ ]:
seed=1493
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.1,random_state=seed)

In [ ]:
##########LightGBM gridsearch CV for best hyperparameter##########
model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt'],
#  'learning_rate':[1],
  'n_estimators':[50],
 'subsample':[0.4],
 'max_depth':[5],
 'reg_alpha':[0.01],
 'reg_lambda':[0]
}
LGBM=gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

In [ ]:
##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
 'booster':['gbtree'],
  'learning_rate':[0.25],
 'n_estimators':[50],
 'max_depth':[11],
 'subsample':[0.7],
 'reg_alpha':[0.01],
 'reg_lambda':[0.01]
}
XG=gridsearch(model_XGBClassifier,param_xg,'XGBoost')

In [ ]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0,bootstrap_type='Bernoulli')
param_cat = {
 'learning_rate':[0.25],
 'n_estimators':[50],
 "boosting_type":["Plain"],
 'max_depth':[7],
 'subsample':[0.9],
    'reg_lambda':[0.01]
}
CAT=gridsearch(model_CatClassifier,param_cat,'CatBoost')

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
      'n_estimators':[50],
     'criterion':['entropy'],
     'max_depth': [9],
     'max_features':['auto']
}
RF=gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')

In [ ]:
shap_plot_interaction(LGBM,algorithm_name="LightGBM",interacted_features=['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
pdp_plot_2d(LGBM,['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
shap_plot_interaction(XG,algorithm_name="XGBoost",interacted_features=['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
pdp_plot_2d(XG,['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
shap_plot_interaction(CAT,algorithm_name="CatBoost",interacted_features=['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
pdp_plot_2d_XG_CAT(CAT,['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
shap_plot_interaction(RF,algorithm_name="Random Forest",interacted_features=['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
pdp_plot_2d(RF,['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])